In [ ]:
import warnings
import random
warnings.filterwarnings('ignore')
import numpy as np
import os
import pandas as pd
import cv2
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
import matplotlib.pyplot as  plt
from sklearn.metrics import confusion_matrix, classification_report
from collections import OrderedDict
import torchvision.models as  models
import itertools
import math
from torch.optim import lr_scheduler
from mtcnn.mtcnn import MTCNN
from PIL import Image
torch.manual_seed(0)


In [ ]:
# #Building a MTCNN object 
# detector = MTCNN()
# #Funtion that takes an image and crops the facial part of the image using MTCNN
# def crop_face(x):
#     x = np.array(x)
#     faces = detector.detect_faces(x)
#     if len(faces)==0:
#         return Image.fromarray(x)
#     for face in faces:
#         x1, y1, width, height = face['box']
#         x2, y2 = x1 + width, y1 + height
#         if x1<0:
#             x1=0
#         if y1 < 0:
#             y1=0
#         cropped_face = x[y1:y2,x1:x2]
#         cropped_face = Image.fromarray(cropped_face.astype('uint8'))
#         return cropped_face
# #Directory where we have the original images
# data_dir = 'Subset For Assignment SFEW'
# for i in os.listdir(data_dir):
#     for j in os.listdir(data_dir+'/'+i):
#         img = Image.open(data_dir+'/'+i+'/'+j)
#         #cropping faces of all the images in the directory
#         crop_faces = crop_face(img)
#         if not os.path.exists('new_data'+'/'+i):
#             #making new directory to keep cropped images 
#             os.makedirs('new_data'+'/'+i)
#         crop_faces.save('new_data'+'/'+i+'/'+j)
        

In [ ]:
path = ('cropped_SFEW')
# Performing data augmentation before sending it to the Network
TRANSFORM_IMG = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.RandomRotation(30),
    transforms.Resize((224,224)),#Standard ImageNet
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))#standard ImageNet
    ])
#Transforming all the images using ImageFolder
full_dataset = torchvision.datasets.ImageFolder(root=path,transform=TRANSFORM_IMG)

In [ ]:
train_size = int(0.80* len(full_dataset)) # 80% for training
valid_size =  int(0.10* len(full_dataset))# 10% for validation
test_size = int(0.101* len(full_dataset)) # 10% for testing
train_dataset,valid_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size,valid_size, test_size])


In [ ]:

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers = 0)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=False,num_workers = 0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers = 0)

In [ ]:
#Plotting sample images that are fed to the network
samples, labels = iter(train_loader).next()
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:24])
np_grid_imgs = grid_imgs.numpy()
plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))

In [ ]:
#5 pretrained models were used .For chosing any other model the final pooled neurons number should 
#be changed according to the mdoel used. 


#base_model = models.resnet152(pretrained = True)
base_model = models.alexnet(pretrained = True)
#base_model = models.vgg13(pretrained = True)
#base_model = models.resnet50(pretrained = True)
#base_model = models.resnet18(pretrained = True)

#Net that uses all the layers except the final layer of the pretrained models and fine tunes the fully connected layers.
class FineTuneModel(nn.Module):
    def __init__(self, base_model):
        super(FineTuneModel, self).__init__()
        # Everything except the last linear layer
        self.features = nn.Sequential(*list(base_model.children())[:-1])
        self.classifier = nn.Linear(9216,200)
        self.fc = nn.Linear(200,7)
        self.drop_out = nn.Dropout(0.5)
        # Freeze those weights
        for p in self.features.parameters():
            p.requires_grad = False


    def forward(self, x):
        f = self.features(x)        
        f = f.view(f.size(0), -1)
        y = self.classifier(f)
        cl_out = torch.sigmoid(y)
        drop = self.drop_out(cl_out)
        f = self.fc(drop)
        return f,cl_out





In [ ]:
model= FineTuneModel(base_model)

# Own implementation of CNN 
# for 1 convoutional layer self.layer2 has to be removed  


# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.layer1 = nn.Sequential(
#             nn.Conv2d(3,16, kernel_size=5, stride=1, padding=2),
#             nn.BatchNorm2d(16),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Dropout2d(0.5))
#         self.layer2 = nn.Sequential(
#             nn.Conv2d(16, 32, kernel_size=7, stride=1, padding=3),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Dropout2d(0.5))
#         self.fc = nn.Linear(74*74*32,100)
#         self.drop = nn.Dropout(0.4)
#         self.fc1=nn.Linear(100,7)

#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = out.reshape(out.size(0), -1)
#         out = self.fc(out)
#         out = torch.tanh(out)
#         out = self.drop(out)
#         out = self.fc1(out)
#         return out

#  Create an instance of the model class 
#model = Net()
#Find total parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

In [ ]:
#Different learning rate and scheduler and optimizers were tried.

criterion = nn.CrossEntropyLoss()
# plist = [
#         {'params': model.classifier.parameters(), 'lr': 5e-3,
#          'params1' : model.fc.parameters(),'lr':5e-3}
#         ]
#optimizer = optim.Adam(model.parameters(), lr=0.001)
#lr_sch = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay = 1e-5)
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)

In [ ]:
# Set the model to training mode
if (torch.cuda.is_available()):
  model=model.cuda()
num_epochs = 50
all_train_losses=[]
all_val_loss = []
for epoch in range(num_epochs):

    train_loss = 0
    model.train()
    hidden = []
    # Process the images in batches
    for batch_idx, (data, target) in enumerate(train_loader):
        # Use the CPU or GPU as appropriate
        if (torch.cuda.is_available()):

          data, target = data.cuda(), target.cuda()
        
        # Reset the optimizer
        optimizer.zero_grad()
        
        # Push the data forward through the model layers
        output,h1= model(data)
        hidden.append(h1)
        # Get the loss
        loss = criterion(output,target)

        train_loss += loss.item()

        # Backpropagate
        loss.backward()
        optimizer.step()
        #lr_sch.step()
    # return average loss for the epoch
    res = torch.cat(hidden,dim=0)
    avg_loss = train_loss / (batch_idx+1)
    all_train_losses.append(avg_loss)
    with torch.no_grad():
        correct = 0
        total = 0
        val_loss = 0
        #all_val_loss = []
        for batch,(images, labels) in enumerate(valid_loader):


            if (torch.cuda.is_available()):
              images, labels = images.cuda(), labels.cuda()


            outputs,h_test= model(images)
            v_loss = criterion(outputs,labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_loss+=v_loss.item()
    avg_val_loss = val_loss/(batch+1)
    all_val_loss.append(avg_val_loss)
    print('Epoch : {}/{} Training set: Average loss: {:.2f} val_loss : {:.2f} val_accuracy : {:.2f}'.format(epoch+1 , num_epochs,avg_loss,avg_val_loss,100*correct/total))



In [ ]:
model.eval()
# Model allocated to evaluation mode . No gradient update from here. 
with torch.no_grad():
    correct = 0
    total = 0
    pred = []
    label= []
    #Setting seed to 0 as dataloader allocates random sequences of input.
    torch.backends.cudnn.deterministic = True
    random.seed(0)
    for images, labels in test_loader:
        if (torch.cuda.is_available()):

          images, labels = images.cuda(), labels.cuda()
        outputs,h= model(images)
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted)
        label.append(labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    l_ = torch.cat(label)
    pred_ = torch.cat(pred)
    print('Test Accuracy of the model on the  test images: {:.2f} %'.format(100 * correct / total))

In [ ]:
#class wise precision recall and f1-score
print("Evaluation of our model on test set")
testing_pred = pred_.cpu()
actual = l_.cpu()
target_names = ['Angry','Disgust','Fear','Happy','Neutral','Sad','Surprise']
print(classification_report(testing_pred,actual,target_names=target_names))

In [ ]:

# Extracting the hidden layer output to perform pruning
k_array=np.array(res.cpu().detach())
scaler = MinMaxScaler()
pattern_vector=k_array.T
pattern_vector=scaler.fit_transform(pattern_vector)
pattern_vector=pattern_vector -0.5
pattern_vector = pattern_vector.tolist()

#finding pairs (combinations) of the pattern vector
def find_tuples_index(lst,num=2):
     return [i for i in itertools.combinations(enumerate(lst), num)]
pair_angle=find_tuples_index(pattern_vector,num=2)
#Function to compute angle

def find_angle(vector1,vector2): 
    vector_1=torch.tensor(vector1 , dtype = torch.float)
    vector_2=torch.tensor(vector2 , dtype = torch.float)
    unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    return math.degrees(angle)

angles = []
indices = []
main =[]
for i in range(len(pair_angle)):
    #finding angles between the pairs in pair_angle
    angles.append(find_angle(pair_angle[i][0][1],pair_angle[i][1][1]))
    #indices of the angle 
    indices.append((pair_angle[i][0][0],pair_angle[i][1][0]))
for j in range(len(angles)):
    main.append((angles[j] , indices[j]))
main=np.array(main)
#creating list for appending all similar pairs index
similar=[]
for k in range(main.shape[0]):
    if main[k][0] <= 15:
        similar.append((main[k][0] ,main[k][1]))
#creating list for appending all cpmplementary pairs index
complimentary=[]
for l in range(main.shape[0]):
    if main[l][0] >= 165:
        complimentary.append((main[l][0] ,main[l][1]))
print(len(complimentary))
print(len(similar))

In [ ]:
#cloning the weights for calculation
weights_new = model.classifier.weight.data
similar_weights = weights_new.clone()
out_weights = model.fc.weight.data
old_out_weights = out_weights.clone()
bias  = model.classifier.bias.data
old_bias = bias.clone()
print("Removing complementary neurons and adding similar weights\n .......")
# creating a list for the index of neurons that are complementary and needs to be removed
to_remove_comp = []
for i in complimentary:
    if i[1][0] not in to_remove_comp:
        to_remove_comp.append(i[1][0])
    if i[1][1] not in to_remove_comp:
        to_remove_comp.append(i[1][1])
#Changing the rows of weight matrix and bais for the corresponding neurons to zero and the column of output weights to zero
for i in to_remove_comp:
    weights_new[i]=0.0
    out_weights[:,i]=0.0
    bias[i]=0.0

#Adding weights of the similar indices and then removing one of them
for i in similar:
    weights_new[i[1][1]]+=similar_weights[i[1][0]]
    out_weights[:,i[1][1]]+=old_out_weights[:,i[1][0]]
    bias[i[1][1]]+=old_bias[i[1][0]]
#After adding removing the neurons for which we have added the weight to another 
for i in similar:
    weights_new[i[1][0]] = 0
    out_weights[:,i[1][0]]=0
    bias[i[1][0]]=0

In [ ]:
c=0
for i in range(weights_new.shape[0]):
    if weights_new[i].cpu().detach().numpy().all()== 0:
        c+=1
c

In [ ]:
model.eval() 
with torch.no_grad():
    correct_r= 0
    total_r = 0
    pred_r = []
    label_r= []
    set_seed(0)
    for images_r, labels_r in test_loader:
        if (torch.cuda.is_available()):

          images_r, labels_r = images_r.cuda(), labels_r.cuda()
        outputs_r,h_r= model(images_r)
        _, predicted_r = torch.max(outputs_r.data, 1)
        pred_r.append(predicted_r)
        label_r.append(labels_r)
        total_r += labels_r.size(0)
        correct_r+= (predicted_r == labels_r).sum().item()
    l_r = torch.cat(label_r)
    pred_r= torch.cat(pred_r)
    print('Test Accuracy of the model on the  test images: {:.2f} %'.format(100 * correct_r / total_r))

In [ ]:
#class wise precision recall and f1-score
print("Evaluation of our model on test set after pruning")
testing_pred = pred_r.cpu()
actual = l_r.cpu()
target_names = ['Angry','Disgust','Fear','Happy','Neutral','Sad','Surprise']
print(classification_report(testing_pred,actual,target_names=target_names))

In [ ]:
weights_new

In [ ]:
model.classifier.weight.data